In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time
from datetime import date

In [11]:
def get_emails(url, pages):

    USERNAME = ''
    PASSWORD = ''

    links = []
    links2 = []
    pageStart = 1

    names = ['Date Added', 'First Name', 'Last Name', 'Position', 'Company', 'Email']
    df = pd.DataFrame(columns = names)

    failures = []
    count = 0

    driver = webdriver.Chrome(executable_path = 'C:\Drivers\Chrome_driver\chromedriver')
    driver.get('https://www.linkedin.com/login?fromSignIn=true&trk=guest_homepage-basic_nav-header-signin')
    driver.maximize_window()

    user_name_box = driver.find_element_by_id('username')
    user_name_box.send_keys(USERNAME)

    password_box = driver.find_element_by_id('password')
    password_box.send_keys(PASSWORD)

    submit_box = driver.find_element_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "from__button--floating", " " ))]') 
    submit_box.click()

    while pageStart < pages + 1:
        driver.get(url + str(pageStart))
        time.sleep(1)
        data = driver.page_source
        start = [m.start() for m in re.finditer('https://www.linkedin.com/in/', data)]
        end = [m.start() for m in re.finditer('","title"', data)]
        for i in range(len(start)):
            links.append(data[start[i]:end[i]])
            links2.append(data[start[i]:end[i]] + '/detail/contact-info/')
        pageStart += 1

    for i in range(len(links)):
        driver.get(links[i])
        try:
            element_present = EC.presence_of_element_located((By.XPATH, '//*[contains(concat( " ", @class, " " ), concat( " ", "break-words", " " ))]'))
            WebDriverWait(driver, 3).until(element_present)
            headline = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "break-words", " " ))]')
            d = headline[1].text
            if d.find('at') != -1:
                d = d.split(' at ')
            else:
                d = d.split(' @ ')
            if len(d) > 1:
                df.at[count, 'Company']= d[1]
            df.at[count, 'Position'] = d[0]
        except:
            failures.append(links[i])
        driver.get(links2[i])
        try:
            element_present = EC.presence_of_element_located((By.XPATH, '//*[(@id = "pv-contact-info")]'))
            WebDriverWait(driver, 3).until(element_present)
            name = driver.find_element_by_xpath('//*[(@id = "pv-contact-info")]')
            email = driver.find_elements_by_xpath('//*[contains(concat( " ", @class, " " ), concat( " ", "t-14", " " ))]')
            df.at[count, 'Date Added'] = date.today()
            c = name.text
            c = c.split(' ')
            df.at[count, 'First Name'] = c[0]
            df.at[count, 'Last Name'] = c[1]
            a = []
            for i in email:
                a.append(i.text)
            b = []
            for i in a:
                if i.find('@') != -1:
                    b.append(i)
            if len(b) > 0:
                df.at[count, 'Email'] = b[0]
        except:
            failures.append(links2[i])
        count += 1
    driver.close()
    df.to_csv('emails.csv', index=False)
    return df

In [12]:
url = 'https://www.linkedin.com/search/results/people/?facetGeoRegion=%5B%22us%3A732%22%5D&facetNetwork=%5B%22F%22%5D&keywords=finance%20director&origin=GLOBAL_SEARCH_HEADER&page='
get_emails(url, 69)

[]

In [ ]:
failures

In [13]:
df

,Date Added,First Name,Last Name,Email
0,2020-05-08,Shubham,Gupta,shubh.gupta2018@gmail.com
1,2020-05-08,Shreya,Raman,shreyaraman99@gmail.com
2,2020-05-08,Shruti,Verma,connect.shrutiv@gmail.com
3,2020-05-08,Justin,Donovan,justin1donovan@gmail.com
4,2020-05-08,Zoe,Husted,zfhusted@gmail.com
...,...,...,...,...
684,2020-05-08,Dominic,Grupalo,dcgrupalo@gmail.com
685,2020-05-08,Kaushal,Patel,khpatel1222@gmail.com
686,2020-05-08,Dr.,Piyush,mykar2un@gmail.com
687,2020-05-08,Sandra,López,l_sandra21@yahoo.com
